[View in Colaboratory](https://colab.research.google.com/github/eventia/deepLearningOpenLecture/blob/master/TwoLayerNet.ipynb)

In [7]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
# This takes several seconds.
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive/Colab Notebooks/"

ch00_hello_world.ipynb	       example.txt		  TrialFunction
ch01_total의 사본	       gradientMethod2d.ipynb	  TwoLayerNet.ipynb
ch01_total.ipynb	       Learn_argmax.ipynb	  Untitled0.ipynb
ch01_total.py.gdoc	       LectureOfMachineLearning   Untitled1.ipynb
ch02_gates.ipynb	       log_graph		  Untitled2.ipynb
ch03_mnist.ipynb	       log_graph의 사본	  Untitled3.ipynb
CH05_BatAffineLayer.ipynb      ML_affine_p174의 사본  Untitled4.ipynb
ColaborCodeSnippet	       ML_affine_p174.ipynb	  Untitled5.ipynb
ColaborCodeSnippet의 사본  mnist_cnn2.py		  Untitled6.ipynb
DeepLearningFromScratch        numerical_diff.ipynb	  Untitled7.ipynb
DemoColaboratory	       result.txt
Drive FUSE example.ipynb       test2Colab.gsheet


In [1]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
# from common.layers import *
# from common.gradient import numerical_gradient
# from collections import OrderedDict




ModuleNotFoundError: ignored

In [0]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads